In [1]:
from utils import *

In /opt/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /opt/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/anaconda3/en

In [3]:
names_to_smiles_file = "../../data/name_to_smiles_dicts/ext_names_to_smiles.txt"
reaction_database_file = "../../data/BKMS-db.xlsx" #Excel format of database file
# reaction_database_file = '../preprocessing_data/cleaned_reactions_of_MetaCyc.txt'

names_to_smiles_tab = pd.read_csv(names_to_smiles_file, sep = "\t", header = None) #import the table of chemical names and associated SMILES string
db = pd.read_excel(reaction_database_file)
# db = pd.read_csv(reaction_database_file, sep = '\t')

#remove chemicals with no SMILES from the table
names_to_smiles_tab = names_to_smiles_tab.dropna()
parsed_brenda_names = open('../../data/name_to_smiles_dicts/brenda_compounds_to_standardized_smiles.json')
parsed_metacyc_names = pd.read_json('../../data/name_to_smiles_dicts/cleaned_metacyc_names_to_smiles.json')

brenda_dict = json.load(parsed_brenda_names)
metacyc_dict = pd.Series(parsed_metacyc_names['SMILES'].values, index=parsed_metacyc_names['Compound'].values).to_dict()

#make a dictionary with (chemical name:SMILES string)
dic = brenda_dict
dic.update(pd.Series(names_to_smiles_tab[1].values,index=names_to_smiles_tab[0]).to_dict())
dic.update(metacyc_dict)
dic[" + "] = "."
dic[" = "] = ">>"
dic[" <=> "] = ">>"
print(dic["glucose"]+dic[" + "]+dic["oxygen"]+dic[" = "]+dic["CO2"])

assert ('Reaction' in db.columns)

/opt/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  


C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O.O=O>>C(=O)=O


In [4]:
#### NOTE: added an additional processing step (removing `^[0-9]+ ` from keys) which was not included in the original processing of the database for 7Apr2021...

brenda_dict = {re.sub("^[Aa]n? |^n |^[0-9]+ ", "", k.lower()).strip() : standardize_smiles(v) for k,v in brenda_dict.items() if standardize_smiles(v) != None}
metacyc_dict = {re.sub("^[Aa]n? |^n |^[0-9]+ ", "", k.lower()).strip() : standardize_smiles(v) for k,v in metacyc_dict.items() if standardize_smiles(v) != None}
dic = {re.sub("^[Aa]n? |^n |^[0-9]+ ", "", k.lower()) : standardize_smiles(v) for k,v in dic.items() if standardize_smiles(v) != None}

dic[" + "] = "."
dic[" = "] = ">>"
dic[" <=> "] = ">>"

In [5]:
print ("Total number of reactions in the BKMS flatfile:", len(db))

Total number of reactions in the BKMS flatfile: 37225


In [6]:
neutralize_reaction('C(C1(\\N\\C=C(CCC(=O)[O-])/C(\\CC(=O)[O-])=1))[NH3+].C(C1(\\N\\C=C(CCC(=O)[O-])/C(\\CC(=O)[O-])=1))[NH3+].C(C1(\\N\\C=C(CCC(=O)[O-])/C(\\CC(=O)[O-])=1))[NH3+].C(C1(\\N\\C=C(CCC(=O)[O-])/C(\\CC(=O)[O-])=1))[NH3+]>>O=C(O)CCc1c[nH]c(Cc2[nH]c(Cc3[nH]c(Cc4[nH]c(CO)c(CC(=O)O)c4CCC(=O)O)c(CC(=O)O)c3CCC(=O)O)c(CC(=O)O)c2CCC(=O)O)c1CC(=O)O')

'NCc1[nH]cc(CCC(=O)O)c1CC(=O)O.NCc1[nH]cc(CCC(=O)O)c1CC(=O)O.NCc1[nH]cc(CCC(=O)O)c1CC(=O)O.NCc1[nH]cc(CCC(=O)O)c1CC(=O)O>>O=C(O)CCc1c[nH]c(Cc2[nH]c(Cc3[nH]c(Cc4[nH]c(CO)c(CC(=O)O)c4CCC(=O)O)c(CC(=O)O)c3CCC(=O)O)c(CC(=O)O)c2CCC(=O)O)c1CC(=O)O'

In [7]:
dic_f = {}
dic_r = {}
dic_f = make_cofactor_dict(np.array(db["Reaction"][:]), 10, 0.9, "f")
dic_r = make_cofactor_dict(np.array(db["Reaction"][:]), 10, 0.9, "r")

cofs = mergeDict(dic_f, dic_r)
print(cofs)

#manually add in list of chemicals to be ignored when converting to SMILES, should be all lowercase
cofs["ignore"] = ["h+", "e-",  "hnu", "water", 'sodium'] #, "h2o", 
#                   "oxygen", "o2", "adp", "amp", "phosphate", 
#                   "diphosphate", "co2", 'sodium', 'na+', 
                  #'coa', 'acyl-coa', 'acetyl-coa', 'succinyl-coa', #keeping coa in for now, removing after mapping
#                   'ammonium', 'nh3', "atp"]

cofs = {k:v for k,v in cofs.items() if v!=['h2o']} #removes some bad "cofactor pairs like glucose:h2o

{'oxidized unknown electron carrier': ['reduced unknown electron carrier'], 'udp-beta-l-rhamnose': ['udp'], 'methylmalonyl-coa': ['h2o', 'co2'], '(2e,6e)-farnesyl diphosphate': ['diphosphate'], 'dtdp-beta-l-rhamnose': ['dtdp'], 'reduced thioredoxin': ['oxidized thioredoxin'], 'udp-alpha-d-glucuronate': ['udp'], 'gdp-d-mannose': ['gdp'], 'caffeoyl-coa': ['coa'], 'cdp-choline': ['cmp'], 'oxidized c-type cytochrome': ['h+', 'reduced c-type cytochrome'], 'electron-transfer quinone': ['electron-transfer quinol'], "3'-phosphoadenylyl-sulfate": ["adenosine 3,5'-bisphosphate"], 'na+': ['na+'], 'biotin': ['diphosphate'], '[protein]-npi-phospho-l-histidine': ['[protein]-l-histidine'], 'prenyl diphosphate': ['diphosphate'], 'thioredoxin': ['thioredoxin disulfide'], 'ah2': ['h2o', 'a'], 'primary alcohol': ['aldehyde'], 'cmp-n-acetyl-beta-neuraminate': ['cmp'], 'udpglucose': ['udp'], 'gdp-l-fucose': ['gdp'], 'electron-transfer flavoprotein': ['reduced electron-transfer flavoprotein'], '[reduced nad

In [8]:
no_smiles_cofs = {k : vs for k,vs in cofs.items() if k not in dic.keys() or np.any([v not in dic.keys() for v in vs])}
cofs = no_smiles_cofs

In [9]:
cofs

{'oxidized unknown electron carrier': ['reduced unknown electron carrier'],
 'reduced thioredoxin': ['oxidized thioredoxin'],
 'oxidized c-type cytochrome': ['h+', 'reduced c-type cytochrome'],
 'electron-transfer quinone': ['electron-transfer quinol'],
 "3'-phosphoadenylyl-sulfate": ["adenosine 3,5'-bisphosphate"],
 '[protein]-npi-phospho-l-histidine': ['[protein]-l-histidine'],
 'thioredoxin': ['thioredoxin disulfide'],
 'electron-transfer flavoprotein': ['reduced electron-transfer flavoprotein'],
 '[reduced nadph-hemoprotein reductase]': ['[oxidized nadph-hemoprotein reductase]',
  'h2o'],
 '[reduced flavoprotein]': ['[oxidized flavoprotein]'],
 'reduced unknown electron carrier': ['oxidized unknown electron carrier'],
 '[reduced nadph---hemoprotein reductase]': ['[oxidized nadph---hemoprotein reductase]',
  'h2o'],
 '[hpr protein]-npi-phospho-l-histidine': ['[hpr]-l-histidine'],
 'malonyl-[acp]': ['co2', 'soluble [acyl-carrier protein]'],
 'sulfur-(sulfur carrier)': ["5'-deoxyadeno

In [10]:
#Example reactions
print(conv_to_smiles("nitric oxide + ammonium + H+ + e- = hydrazine + H2O", dic, cofs))
print(conv_to_smiles("nitric oxide + ammonium + H+ + e- = hydrazine + H2O", 
                     [metacyc_dict, brenda_dict, dic], cofs))

print(conv_to_smiles("Diphosphate", dic, cofs))
print(conv_to_smiles("nitric oxide + ammonium + H+ + e- = hydrazine + H2O", 
                     [metacyc_dict, brenda_dict, dic], cofs))

print(conv_to_smiles("carboxylate", dic, cofs))
print(conv_to_smiles("S-acylglutathione + H2O = glutathione + a carboxylate", dic, cofs))
print(conv_to_smiles("n CDP-ribitol + 4-O-[1-D-ribitylphospho-(2R)-1-glycerophospho]-N-acetyl-beta-D-mannosaminyl-(1->4)-N-acetyl-alpha-D-glucosaminyl-diphospho-ditrans,octacis-undecaprenol = n CMP + 4-O-[(1-D-ribitylphospho)n-(1-D-ribitylphospho)-(2R)-1-glycerophospho]-N-acetyl-beta-D-mannosaminyl-(1->4)-N-acetyl-alpha-D-glucosaminyl-diphospho-ditrans,octacis-undecaprenol", dic, cofs))
print(conv_to_smiles("ent-sandaracopimaradiene + [reduced NADPH-hemoprotein reductase] + O2 = ent-sandaracopimaradien-3beta-ol + [oxidized NADPH-hemoprotein reductase] + H2O", dic, cofs))
print(conv_to_smiles("(S)-4-hydroxymandelate + O2 = 4-hydroxybenzaldehyde + CO2 + H2O2", dic, cofs))

print(conv_to_smiles("(S)-4-hydroxymandelate + O2 = 4-hydroxybenzaldehyde + CO2 + H2O2", 
                     [metacyc_dict, brenda_dict, dic], cofs))

[N]=O.[NH4+]>>NN.O
N=O.[NH4+]>>NN.O
O=P([O-])([O-])OP(=O)([O-])[O-]
N=O.[NH4+]>>NN.O
*C(=O)[O-]
*C(=O)SC[C@H](NC(=O)CC[C@H]([NH3+])C(=O)[O-])C(=O)NCC(=O)[O-].O>>N[C@@H](CCC(=O)N[C@@H](CS)C(=O)NCC(=O)O)C(=O)O.*C(=O)[O-]
Nc1ccn([C@@H]2O[C@H](COP(=O)([O-])OP(=O)([O-])OC[C@@H](O)[C@@H](O)[C@@H](O)CO)[C@@H](O)[C@H]2O)c(=O)n1.CC(=O)N[C@@H]1[C@H](OC2[C@@H](CO)O[C@H](OP(=O)([O-])OP(=O)([O-])OC/C=C(/C)CC/C=C(/C)CC/C=C(/C)CC/C=C(/C)CC/C=C(/C)CC/C=C(/C)CC/C=C(/C)CC/C=C(/C)CC/C=C(\C)CC/C=C(\C)CCC=C(C)C)[C@H](NC(C)=O)[C@H]2O)O[C@H](CO)[C@@H](OP(=O)([O-])OC[C@H](O)COP(=O)([O-])OC[C@@H](O)[C@@H](O)[C@@H](O)CO)[C@@H]1O>>Nc1ccn([C@@H]2O[C@H](COP(=O)([O-])[O-])[C@@H](O)[C@H]2O)c(=O)n1.CC(=O)N[C@@H]1[C@H](O[C@@H]2[C@@H](CO)O[C@H](OP(=O)(O)OP(=O)(O)OC/C=C(/C)CC/C=C(/C)CC/C=C(/C)CC/C=C(/C)CC/C=C(/C)CC/C=C(/C)CC/C=C(/C)CC/C=C(/C)CC/C=C(\C)CC/C=C(\C)CCC=C(C)C)[C@H](NC(C)=O)[C@H]2O)O[C@H](CO)[C@@H](OP(=O)(O)OC[C@H](O)COP(=O)(O)OC[C@@H](O)[C@@H](O)[C@@H](O)COP(=O)(O)OC[C@@H](O)[C@@H](O)[C@@H](O)CO)[C@@H]1O
C=C

In [11]:
#example reactions
print(conv_to_smiles("4-hydroxybenzaldehyde = 4-hydroxybenzaldehyde + NADP+", [dic], cofs))

print(conv_to_smiles("4-hydroxybenzaldehyde + NADPH = 4-hydroxybenzaldehyde", [dic], cofs))

print(conv_to_smiles("NADPH + 4-hydroxybenzaldehyde = 4-hydroxybenzaldehyde + NADP+", [dic], cofs))

print(conv_to_smiles("NADPH + 4-hydroxybenzaldehyde + CoA = 4-hydroxybenzaldehyde + NADP+ + acyl-CoA", [dic], cofs))

print(conv_to_smiles("NADPH = NADP+", [dic], cofs))

O=Cc1ccc(O)cc1>>O=Cc1ccc(O)cc1.N=c1[nH]cnc2c1ncn2[C@@H]1O[C@H](CO[PH](=O)(=O)O[PH](=O)(=O)OC[C@H]2O[C@@H]([n+]3cccc(C(N)=O)c3)[C@H](O)[C@@H]2O)[C@@H](O)[C@H]1O[PH](=O)(=O)O
O=Cc1ccc(O)cc1.N=c1[nH]cnc2c1ncn2[C@@H]1O[C@H](CO[PH](=O)(=O)O[PH](=O)(=O)OC[C@H]2O[C@@H](N3C=CCC(C(N)=O)=C3)[C@H](O)[C@@H]2O)[C@@H](O)[C@H]1O[PH](=O)(=O)O>>O=Cc1ccc(O)cc1
N=c1[nH]cnc2c1ncn2[C@@H]1O[C@H](CO[PH](=O)(=O)O[PH](=O)(=O)OC[C@H]2O[C@@H](N3C=CCC(C(N)=O)=C3)[C@H](O)[C@@H]2O)[C@@H](O)[C@H]1O[PH](=O)(=O)O.O=Cc1ccc(O)cc1>>O=Cc1ccc(O)cc1.N=c1[nH]cnc2c1ncn2[C@@H]1O[C@H](CO[PH](=O)(=O)O[PH](=O)(=O)OC[C@H]2O[C@@H]([n+]3cccc(C(N)=O)c3)[C@H](O)[C@@H]2O)[C@@H](O)[C@H]1O[PH](=O)(=O)O
N=c1[nH]cnc2c1ncn2[C@@H]1O[C@H](CO[PH](=O)(=O)O[PH](=O)(=O)OC[C@H]2O[C@@H](N3C=CCC(C(N)=O)=C3)[C@H](O)[C@@H]2O)[C@@H](O)[C@H]1O[PH](=O)(=O)O.O=Cc1ccc(O)cc1.CC(C)(CO[PH](=O)(=O)O[PH](=O)(=O)OC[C@H]1O[C@@H](n2cnc3c(=N)[nH]cnc32)[C@H](O)[C@@H]1O[PH](=O)(=O)O)C(=O)C(O)NCCC(=O)NCCS>>O=Cc1ccc(O)cc1.N=c1[nH]cnc2c1ncn2[C@@H]1O[C@H](CO[PH](=O)(=O)O

In [12]:
db = pd.read_excel(reaction_database_file)
db = db.dropna(subset=['Reaction'])
db["smiles"] = db["Reaction"].map(lambda x: neutralize_reaction(conv_to_smiles(x.lower(), [metacyc_dict, brenda_dict, dic], cofs)))

/opt/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  """Entry point for launching an IPython kernel.


In [13]:
# with previous method: 27110
db.dropna(subset=['smiles'])

,ID,EC_Number,Recommended_Name,Reaction,Reaction_ID_BRENDA,Reaction_ID_KEGG,Reaction_ID_MetaCyc,Reaction_ID_SABIO_RK,BRENDA_Pathway_Name,KEGG_Pathway_ID,KEGG_Pathway_Name,MetaCyc_Pathway_ID,MetaCyc_Pathway_Name,Stoichiometry_Check,Missing_Substrate,Missing_Product,Commentary_KEGG,Commentary_MetaCyc,Remark,smiles
0,1,5.3.3.3,vinylacetyl-CoA DELTA-isomerase,vinylacetyl-CoA = (E)-but-2-enoyl-CoA,BR1,R03031,VINYLACETYL-COA-DELTA-ISOMERASE-RXN,NaN,NaN,rn00650,Butanoate metabolism,NaN,NaN,NaN,NaN,NaN,NaN,NaN,natural substrates,C=CCC(=O)SCCNC(=O)CCNC(=O)[C@H](O)C(C)(C)COP(=...
1,2,6.3.2.1,pantoate-beta-alanine ligase (AMP-forming),ATP + (R)-pantoate + beta-alanine = AMP + diph...,BR101,R02473,PANTOATE-BETA-ALANINE-LIG-RXN,2406,pantothenate biosynthesis,rn00410; rn00770; rn01110,beta-Alanine metabolism; Pantothenate and CoA ...,PANTO-PWY,phosphopantothenate biosynthesis I,NaN,NaN,NaN,NaN,NaN,natural substrates,CC(C)(CO)[C@@H](O)C(=O)O.NCCC(=O)O.Nc1ncnc2c1n...
2,3,6.3.2.10,UDP-N-acetylmuramoyl-tripeptide-D-alanyl-D-ala...,ATP + UDP-N-acetylmuramoyl-L-alanyl-gamma-D-gl...,BR102,R04573,6.3.2.10-RXN,4244,peptidoglycan biosynthesis,rn00550,Peptidoglycan biosynthesis,PWY-6386,UDP-N-acetylmuramoyl-pentapeptide biosynthesis...,NaN,NaN,NaN,NaN,NaN,natural substrates,CC(=O)N[C@H]1[C@@H](OP(=O)(O)OP(=O)(O)OC[C@H]2...
3,4,6.3.2.4,D-Alanine-D-alanine ligase,ATP + UDP-N-acetylmuramoyl-L-alanyl-gamma-D-gl...,BR102,R04573,6.3.2.10-RXN,4244,peptidoglycan biosynthesis,rn00550,Peptidoglycan biosynthesis,PWY-6386,UDP-N-acetylmuramoyl-pentapeptide biosynthesis...,NaN,NaN,NaN,NaN,NaN,natural substrates,CC(=O)N[C@H]1[C@@H](OP(=O)(O)OP(=O)(O)OC[C@H]2...
4,5,6.3.2.11,carnosine synthase,ATP + L-histidine + beta-alanine = ADP + phosp...,BR103,R01164,CARNOSINE-SYNTHASE-RXN,NaN,NaN,rn00340; rn00410,Histidine metabolism; beta-Alanine metabolism,PWY66-420,carnosine biosynthesis,NaN,NaN,NaN,NaN,NaN,natural substrates,NCCC(=O)O.N[C@@H](Cc1c[nH]cn1)C(=O)O.Nc1ncnc2c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37216,89227,3.4.21.22,coagulation factor IXa,methylsulfonyl-D-cyclohexylglycyl-Arg-7-amido-...,BS410899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,natural substrates,Cc1cc(=O)oc2cc(NC(=O)C(CCCN=C(N)N)NC(=O)CNC(=O...
37217,89228,3.4.21.68,t-Plasminogen activator,methylsulfonyl-D-cyclohexylglycyl-Arg-7-amido-...,BS410899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,natural substrates,Cc1cc(=O)oc2cc(NC(=O)C(CCCN=C(N)N)NC(=O)CNC(=O...
37219,89245,5.5.1.4,inositol-3-phosphate synthase,D-glucose 6-phosphate = 1D-inositol 3-phosphate,BS368807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,natural substrates,O=C[C@H](O)[C@@H](O)[C@H](O)[C@H](O)COP(=O)(O)...
37223,89262,3.6.1.9,nucleotide diphosphatase,"2',3'-cAMP + H2O = adenosine monophosphate",BS165471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,natural substrates,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO)[C@H]2OP(=O)(O)...


In [14]:
# Uncomment to save the database table with the added smiles

db.to_csv('../../data/processed_bkms_database/bkms_w_SMILES.txt', sep = '\t')